In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!npm install localtunnel


npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.665s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
# Recommender System Imports
# Content-Based Filtering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Collaborative-Based Filtering
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
# To work with text data
import re
import string
import pandas as pd
df = pd.read_csv('1662574418893344.csv')
unique_dishes = df['Name'].unique()
num_dishes = len(unique_dishes)
unique_c_types = df['C_Type'].unique()
unique_veg_non = df['Veg_Non'].unique()
num_rows = len(df)
df_shape = df.shape
df_info = df.info()
def text_cleaning(text):
      cleaned_text = "".join([char for char in text if char not in string.punctuation])
      return cleaned_text
df['Describe_cleaned'] = df['Describe'].apply(text_cleaning)
num_duplicates = df.duplicated().sum()
null_values = df.isnull().sum()
df_description = df.describe()
indices = pd.Series(df.index, index=df['Name']).drop_duplicates()
def get_recommendations(title):
    # Check if the input title exists in the dataset
    if title not in df['Name'].values:
        return "Input not found in the dataset! Please enter a valid food title."

    # Create a TfidfVectorizer object with English stop words
    tfidf = TfidfVectorizer(stop_words='english')

    # Fit and transform the 'Describe' column of the DataFrame
    tfidf_matrix = tfidf.fit_transform(df['Describe_cleaned'])

    # Compute the cosine similarity matrix using the linear_kernel function
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    # Get the index of the input title
    idx = df[df['Name'] == title].index[0]

    # Get the similarity scores for all food
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the food based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the 5 most similar food
    sim_scores = sim_scores[1:6]

    # Get the indices of the food
    food_indices = [i[0] for i in sim_scores]

    # Return the 5 most similar food
    recommendations1 = df.iloc[food_indices].drop(columns=['Food_ID'])
    return recommendations1

features = ['C_Type', 'Veg_Non', 'Describe']
def create_soup(x):
    return x['C_Type'] + " " + x['Veg_Non'] + " " + x['Describe']
df['soup'] = df.apply(create_soup, axis=1)
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])
count_matrix_shape = count_matrix.shape
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
df = df.reset_index(drop=True)
indices = pd.Series(df.index, index=df['Name'])
#recommendations = get_recommendations('tricolour salad', cosine_sim2)

#approach 2
rating = pd.read_csv('ratings.csv')
null_values = rating.isnull().sum()
rating = rating[:511]
rating.isnull().sum()
food_rating = rating.groupby(by = 'Food_ID').count()
food_rating = food_rating['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
user_rating = rating.groupby(by='User_ID').count()
user_rating = user_rating['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
rating_matrix = rating.pivot_table(index='Food_ID',columns='User_ID',values='Rating').fillna(0)
csr_rating_matrix =  csr_matrix(rating_matrix.values)
recommender = NearestNeighbors(metric='cosine')
recommender.fit(csr_rating_matrix)
def Get_Recommendations(title):
    user= df[df['Name']==title]
    if len(user) == 0:
        return "Invalid input! Please enter a valid food title."
    food_id = user['Food_ID'].values[0]

    if food_id not in rating_matrix.index:
        return "Rating data not available for this food. Unable to provide recommendations."
    user_index = np.where(rating_matrix.index==int(user['Food_ID']))[0][0]
    user_ratings = rating_matrix.iloc[user_index]
    reshaped = user_ratings.values.reshape(1,-1)
    distances, indices = recommender.kneighbors(reshaped,n_neighbors=16)
    nearest_neighbors_indices = rating_matrix.iloc[indices[0]].index[1:]
    nearest_neighbors = pd.DataFrame({'Food_ID': nearest_neighbors_indices})
    result = pd.merge(nearest_neighbors,df,on='Food_ID',how='left')
    result = result.drop(columns=['Food_ID'])
    return result.head()
#Get_Recommendations('tricolour salad')

def main():
    st.title("FoodieMatch")

    # Get user inputs
    input1 = st.text_input("Find Your Flavor")
    input2 = st.text_input("Explore Similar Tastes")
    # Check if the user inputs are valid and present in the dataset
    if input1.strip() == "" or input2.strip() == "":
        st.error("Explore dishes from different cuisines and enter your choice.")
    else:
        if input1 not in df['Name'].values:
          st.error("Invalid input! Please refer to the cuisines and try again.")
        if input2 not in df['Name'].values:
          st.error("Invalid input! Please refer to the cuisines and try again.")
    # Process inputs and get recommendations
    if st.button("Get Recommendations"):
        recommendations_approach1 = get_recommendations(input1)
        recommendations_approach2 = Get_Recommendations(input2)

        # Display recommendations
        st.write("Approach 1 Recommendations:", recommendations_approach1)
        st.write("Approach 2 Recommendations:", recommendations_approach2)
    unique_c_types = df['C_Type'].unique()
    unique_veg_non = ['All'] + df['Veg_Non'].unique().tolist()

    # Create a sidebar section for C_Type buttons
    st.sidebar.title("Filters")
    selected_cuisine = st.sidebar.radio("Select Cuisine Type", unique_c_types)
    selected_veg_non = st.sidebar.selectbox("Select Veg/Non-Veg", unique_veg_non)

    # Filter foods based on selected criteria
    if selected_veg_non == "All":
      filtered_foods = df[df['C_Type'] == selected_cuisine]['Name']
    else:
      filtered_foods = df[(df['C_Type'] == selected_cuisine) & (df['Veg_Non'] == selected_veg_non)]['Name']


    # Display filtered foods
    st.header(f"Foods in {selected_cuisine} Cuisine")
    st.write(filtered_foods.drop(columns=['Food_ID']))

if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & curl ipv4.icanhazip.com

34.141.180.128


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.649s
your url is: https://wet-clocks-tan.loca.lt
